In [43]:
# ================================
# 0. 패키지 설치 (필요 시)
# ================================
# !pip install timm tqdm pandas torch

import os
import torch
import torch.nn as nn
import pandas as pd
import timm
from tqdm import tqdm

# ================================
# 1. 경로 및 환경 설정
# ================================
CACHE_DIR   = "./slice_cache0_224"            # 캐시된 환자별 슬라이스 텐서(.pt)
CSV_PATH    = "./clinical.csv"                # 임상 CSV (PatientID 포함)
MODELS_DIR  = "./models"                      # 모델들이 있는 폴더
OUTPUT_DIR  = "./features/test1_1"                    # 결과 저장 폴더
N_RUNS      = 30                              # run 0 ~ 29
BATCH_SIZE  = 16                              # CPU에서 8~32 권장

DEVICE = torch.device("cpu")                  # ✅ CPU 강제
torch.set_num_threads(max(1, os.cpu_count() // 2))  # 노트북 CPU 과부하 방지
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("DEVICE:", DEVICE)
print("CACHE_DIR:", CACHE_DIR)
print("MODELS_DIR:", MODELS_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)

# ================================
# 2. 모델 정의 (BEiT 백본)
# ================================
class BEiTBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model(
            "beit_base_patch16_224", pretrained=False, num_classes=0
        )

    def forward(self, x):
        return self.backbone(x)  # (B, D)

# ================================
# 3. DataFrame 로딩 & PatientID 정리
# ================================
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"임상 CSV를 찾을 수 없습니다: {CSV_PATH}")

df = pd.read_csv(CSV_PATH)

# ✅ PatientID에 _0000 붙여서 캐시 파일과 매칭
df["PatientID"] = df["PatientID"].astype(str) + "_0000"

# 캐시(.pt) 파일 존재하는 환자만 남기기
df = df[df["PatientID"].apply(lambda x: os.path.exists(os.path.join(CACHE_DIR, f"{x}.pt")))]
df = df.reset_index(drop=True)

print(f"[DEBUG] 대상 환자 수: {len(df)}")
if len(df) == 0:
    raise RuntimeError("df가 비었습니다. CSV 또는 캐시(.pt) 경로 확인 필요")

print(df["PatientID"].head())

# ================================
# 4. 여러 모델 반복 실행 (run 0 ~ 29)
# ================================
for run in range(N_RUNS):
    print(f"\n========== Run {run} ==========")
    model_path = os.path.join(MODELS_DIR, f"best_model_beit_run{run}.pt")
    out_csv    = os.path.join(OUTPUT_DIR, f"beit_base_patch16_224_features_{run+1}.csv")

    if not os.path.exists(model_path):
        print(f"⚠️ 모델 파일이 없습니다. 건너뜀 → {model_path}")
        continue

    # ----- 4-1) 모델 로딩 -----
    model = BEiTBackbone().to(DEVICE)
    ckpt = torch.load(model_path, map_location=DEVICE)

    # ckpt 형태에 따라 state_dict 추출
    state_dict = ckpt["state_dict"] if isinstance(ckpt, dict) and "state_dict" in ckpt else ckpt

    # 'backbone.' prefix 정리 및 호환 키만 로딩
    filtered = {
        k.replace("backbone.", ""): v
        for k, v in state_dict.items()
        if k.startswith("backbone.") or k in model.backbone.state_dict()
    }

    missing, unexpected = model.backbone.load_state_dict(filtered, strict=False)
    if missing:
        print("⚠️ missing keys (일부만 표시):", missing[:5], "..." if len(missing) > 5 else "")
    if unexpected:
        print("⚠️ unexpected keys (일부만 표시):", unexpected[:5], "..." if len(unexpected) > 5 else "")

    model.eval()
    print("✅ 모델 로딩 완료:", os.path.basename(model_path))

    # ----- 4-2) Feature 추출 -----
    features = []
    with torch.no_grad():
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Extracting run{run}"):
            pid = row["PatientID"]
            cache_path = os.path.join(CACHE_DIR, f"{pid}.pt")
            if not os.path.exists(cache_path):
                print(f"⚠️ {pid} 캐시 없음 → skip")
                continue

            imgs = torch.load(cache_path, map_location=DEVICE)  # (N, C, 224, 224)
            imgs = imgs.float()

            # 단일 채널이면 3채널로 확장
            if imgs.ndim == 4 and imgs.shape[1] == 1:
                imgs = imgs.repeat(1, 3, 1, 1)

            outs = []
            for start in range(0, imgs.shape[0], BATCH_SIZE):
                batch = imgs[start:start + BATCH_SIZE].to(DEVICE)
                out = model(batch)  # (b, D)
                outs.append(out.cpu())

            feats = torch.cat(outs, dim=0)       # (N, D)
            mean_feat = feats.mean(dim=0).numpy().tolist()
            features.append([pid] + mean_feat)

    if not features:
        print(f"⚠️ 추출된 feature가 없어 저장을 생략합니다 (run {run}).")
        continue

    # ----- 4-3) 저장 -----
    feature_dim = len(features[0]) - 1
    columns = ["PatientID"] + [f"feat_{i}" for i in range(feature_dim)]
    df_feat = pd.DataFrame(features, columns=columns)
    df_feat.to_csv(out_csv, index=False)

    print(f"🎉 저장 완료 → {out_csv}")
    print(f"📊 환자 수: {df_feat.shape[0]}, feature_dim: {feature_dim}")

print("\n✅ 모든 실행이 완료되었습니다.")


DEVICE: cpu
CACHE_DIR: ./slice_cache0_224
MODELS_DIR: ./models
OUTPUT_DIR: ./features/test1_1
[DEBUG] 대상 환자 수: 55
0    TCGA-VS-A8EB_0000
1    TCGA-VS-A8EC_0000
2    TCGA-VS-A8EG_0000
3    TCGA-VS-A8EH_0000
4    TCGA-VS-A8EI_0000
Name: PatientID, dtype: object

========== Run 0 ==========


/var/folders/z1/5g0wf16x3l1_ngkxjnjcsw3w0000gn/T/ipykernel_12970/3098639267.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location=D

✅ 모델 로딩 완료: best_model_beit_run0.pt


Extracting run0:   0%|          | 0/55 [00:00<?, ?it/s]/var/folders/z1/5g0wf16x3l1_ngkxjnjcsw3w0000gn/T/ipykernel_12970/3098639267.py:111: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

🎉 저장 완료 → ./features/test1_1/beit_base_patch16_224_features_1.csv
📊 환자 수: 55, feature_dim: 768

========== Run 1 ==========
✅ 모델 로딩 완료: best_model_beit_run1.pt


...

🎉 저장 완료 → ./features/test1_1/beit_base_patch16_224_features_28.csv
📊 환자 수: 55, feature_dim: 768

========== Run 28 ==========
✅ 모델 로딩 완료: best_model_beit_run28.pt


Extracting run28: 100%|██████████| 55/55 [01:12<00:00,  1.33s/it]


🎉 저장 완료 → ./features/test1_1/beit_base_patch16_224_features_29.csv
📊 환자 수: 55, feature_dim: 768

========== Run 29 ==========
✅ 모델 로딩 완료: best_model_beit_run29.pt


Extracting run29: 100%|██████████| 55/55 [01:13<00:00,  1.34s/it]

🎉 저장 완료 → ./features/test1_1/beit_base_patch16_224_features_30.csv
📊 환자 수: 55, feature_dim: 768

✅ 모든 실행이 완료되었습니다.
